# Criação do banco de dados simulado para o Technee Banco Safra
### Equipe 5 - Safra Hackers

#### Link para download do dataset original:
#### https://www.kaggle.com/ntnu-testimon/paysim1


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('PS_20174392719_1491204439457_log.csv')

#### Removendo as fraudes

In [3]:
df = df[df['isFraud']==0]

#### Os nomes de origem não são significativos, são sempre únicos nesse dataset

In [4]:
df['nameOrig'].drop_duplicates()

0          C1231006815
1          C1666544295
4          C2048537720
5            C90045638
6           C154988899
              ...     
6362319     C642813806
6362320     C691808084
6362321     C102120699
6362322     C614459560
6362323      C49652609
Name: nameOrig, Length: 6345122, dtype: object

#### Os nomes de conta destino já tem contas com mais de uma transferência

In [5]:
df['nameDest'].drop_duplicates()

0          M1979787155
1          M2044282225
4          M1230701703
5           M573487274
6           M408069119
              ...     
6362316     M675916850
6362318    M1126011651
6362319     M747723689
6362320    C1818183087
6362322     C855350324
Name: nameDest, Length: 2719685, dtype: object

### Agrupando a quantidade de transferências pela "conta destino" do dataset

In [6]:
df_count = df.groupby(['nameDest']).count()

### 2.2 Milhões das amostras são de contas que receberam somente 1 transferência em todo dataset
### Não é interessante usar dados de contas com poucas transferências

In [7]:
df_count['amount'].value_counts().head(10)

1     2260777
2       76818
3       57210
4       45030
5       36770
6       30396
7       25760
8       21901
9       18605
10      16310
Name: amount, dtype: int64

### Tipos de transação

In [8]:
df['type'].drop_duplicates()

0       PAYMENT
9         DEBIT
15     CASH_OUT
19     TRANSFER
389     CASH_IN
Name: type, dtype: object

### Vendo perfil das contas que tem mais transferências

In [9]:
df_count.sort_values(by='amount',ascending=False).head(10)

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
nameDest,,,,,,,,,,
C1286084959,113,113,113,113,113,113,113,113,113,113
C985934102,109,109,109,109,109,109,109,109,109,109
C665576141,105,105,105,105,105,105,105,105,105,105
C2083562754,102,102,102,102,102,102,102,102,102,102
C248609774,101,101,101,101,101,101,101,101,101,101
C1590550415,101,101,101,101,101,101,101,101,101,101
C1789550256,99,99,99,99,99,99,99,99,99,99
C451111351,99,99,99,99,99,99,99,99,99,99
C1360767589,98,98,98,98,98,98,98,98,98,98


#### 1205 contas tem pelo menos 50 transferências

In [10]:
df_count[df_count['step']>=50]

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
nameDest,,,,,,,,,,
C1001367899,52,52,52,52,52,52,52,52,52,52
C1003872245,57,57,57,57,57,57,57,57,57,57
C1005928818,55,55,55,55,55,55,55,55,55,55
C1013700132,71,71,71,71,71,71,71,71,71,71
C1014154376,54,54,54,54,54,54,54,54,54,54
...,...,...,...,...,...,...,...,...,...,...
C99509209,58,58,58,58,58,58,58,58,58,58
C997608398,59,59,59,59,59,59,59,59,59,59
C998136421,50,50,50,50,50,50,50,50,50,50


### Selecionando contas com mais de 50 transferências e filtrando as colunas

In [11]:
df2 = df.loc[df['nameDest'].isin(df_count[df_count['step']>=50].index)]

In [12]:
df3 = df2[['nameDest','type','amount','oldbalanceDest','newbalanceDest']]

In [13]:
df3

,nameDest,type,amount,oldbalanceDest,newbalanceDest
10,C997608398,DEBIT,9644.94,10845.00,157982.12
22,C515132998,DEBIT,1065.41,10330.00,0.00
24,C932583850,TRANSFER,311685.89,6267.00,2719172.89
41,C1297685781,DEBIT,5758.59,209699.00,16997.22
47,C824009085,CASH_OUT,56953.90,70253.00,64106.18
...,...,...,...,...,...
6358790,C1821075330,CASH_OUT,202691.57,8382300.59,8584992.16
6359405,C1167432890,CASH_OUT,28593.22,8089356.73,8117949.95
6359621,C814408370,CASH_OUT,216666.53,10119297.16,10335963.70
6360888,C1501536847,CASH_OUT,136062.35,10035444.63,10171506.98


In [20]:
df3['amount'] = df3['amount'] * 100

C:\Users\alexa\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
df3

,nameDest,type,amount,oldbalanceDest,newbalanceDest
10,C997608398,DEBIT,9644.94,10845.00,157982.12
22,C515132998,DEBIT,1065.41,10330.00,0.00
24,C932583850,TRANSFER,311685.89,6267.00,2719172.89
41,C1297685781,DEBIT,5758.59,209699.00,16997.22
47,C824009085,CASH_OUT,56953.90,70253.00,64106.18
...,...,...,...,...,...
6358790,C1821075330,CASH_OUT,202691.57,8382300.59,8584992.16
6359405,C1167432890,CASH_OUT,28593.22,8089356.73,8117949.95
6359621,C814408370,CASH_OUT,216666.53,10119297.16,10335963.70
6360888,C1501536847,CASH_OUT,136062.35,10035444.63,10171506.98


In [25]:
for i in df3['nameDest'].drop_duplicates():
    df3['oldbalanceDest'].at[df3['nameDest']==i] = max(df3[df3['nameDest']==i]['oldbalanceDest'])

C:\Users\alexa\.conda\envs\tensorflow\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\alexa\.conda\envs\tensorflow\lib\site-packages\pandas\core\series.py:1448: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.loc[label] = value


In [51]:
index_list = df3[df3['nameDest']=='C997608398'].index
i = 'C997608398'

str(df3[(df3['nameDest'] == i) & (df3.index == index_list[0])]['type']) in ['DEBIT','CASH_OUT','PAYMENT']

False

In [71]:
df3[df3.index == k]['type'].values

array(['DEBIT'], dtype=object)

In [77]:
from tqdm import tqdm

for i in tqdm(df3['nameDest'].drop_duplicates()):
    
    balance = df3[df3['nameDest']==i]['oldbalanceDest'].head(1)
    
    for k in df3[df3['nameDest']==i].index:
        if df3[df3.index == k]['type'].values in ['DEBIT','CASH_OUT','PAYMENT']:
            df3['oldbalanceDest'].at[df3.index == k] = balance
            df3['newbalanceDest'].at[df3.index == k] = balance - df3[df3.index == k]['newbalanceDest']
            balance = balance - df3[df3.index == k]['newbalanceDest']
        else:
            df3['oldbalanceDest'].at[df3.index == k] = balance
            df3['newbalanceDest'].at[df3.index == k] = balance + df3[df3.index == k]['newbalanceDest']
            balance = balance + df3[df3.index == k]['newbalanceDest']

100%|██████████████████████████████████████████████████████████████████████████████| 1205/1205 [07:20<00:00,  2.73it/s]


In [76]:
df3[df3['nameDest']=='C824009085']

,nameDest,type,amount,oldbalanceDest,newbalanceDest
47,C824009085,CASH_OUT,56953.90,6261276.61,64106.18
181,C824009085,CASH_OUT,57279.11,6261276.61,64106.18
428,C824009085,CASH_IN,147388.67,6261276.61,64106.18
1813,C824009085,TRANSFER,27008.84,6261276.61,64106.18
11618,C824009085,TRANSFER,45819.53,6261276.61,120565.89
14326,C824009085,CASH_IN,365692.36,6261276.61,120565.89
21163,C824009085,CASH_OUT,375262.87,6261276.61,120565.89
23088,C824009085,CASH_IN,45829.87,6261276.61,120565.89
24675,C824009085,CASH_IN,208867.11,6261276.61,120565.89
33141,C824009085,CASH_IN,111388.30,6261276.61,9177.59


In [100]:
df3['amount'] = df3['amount']/100

C:\Users\alexa\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [106]:
df3[df3['nameDest']=='C997608398']['amount'].mean()

2336.8000932203386